# Python SDK configuration

This notebook illustrates how to connect to the Waylay Platform using the Python SDK (waylay-py)

When connecting to the waylay platform, the calls that the SDK makes must be properly authenticated, and use the correct endpoints.

In general
* you authenticate with _client credentials_ in the form of an `apiKey` and `apiSecret` that you have created using the Waylay Console application.
* since version `0.5.0` the configuration relies on a single `gateway_url` (`api.waylay.io` for the Enterprise platform) to indicate the Waylay platform. Legacy clients configure an authorisation server `accounts_url` (`accounts-api.waylay.io` for Enterprise). 
* the client exchanges these _credentials_ for a _waylay token_ with limited time validity, and uses this token to perform backend calls.
* For legacy clients configured with `accounts_url`, the client uses the _global settings_ configurations (managed in the Waylay Console) to define any specific url endpoints.
* Both endpoint configuration methods allow local endpoint overrides which can be saved in configuration profiles.

This notebook illustrates an interactive authentication method that is intended for usage in python notebooks:
* it does not leak secrets into the notebook
* if requested, it stores authentication information on as a _profile_ file in a [platform-specific user config directory](https://pypi.org/project/appdirs/) on the local file system. 

Non-interactive authentication methods (e.g. for integration in scripts) are discussed at the end of the notebook.

The `WaylayClient` object represents a (configured) client that has access to multiple Waylay Services.

The `WaylayConfig` object represents a local client settings object, that can be saved.

In [1]:
from waylay import WaylayConfig, WaylayClient, __version__ as version
version

'v0.7.0'

## Authentication with stored profiles

The Python SDK connection settings can be stored in local profile files.

For this demo, we first delete the 'demo' profile (if it exists)

In [2]:
WaylayConfig.delete('demo')

waylay configuration removed: /Users/thomas/Library/Application Support/Waylay/python_sdk/.profile.demo.json


'/Users/thomas/Library/Application Support/Waylay/python_sdk/.profile.demo.json'

You can authenticate with a profile as follows. When the profile is not found on the file system, it will be created interactively

In [3]:
waylay_client = WaylayClient.from_profile('demo')

Authenticating to the Waylay Platform
Proposed api gateway: https://api.waylay.io
Please confirm, or specify a gateway by platform id, hostname or url.
Examples:
    'enterprise' (or 'api.waylay.io') for the Enterprise platform,
    'https://waylay-api.mycompany.com' as a custom endpoint url
        


> Press enter to confirm, or specify an alternate gateway [https://api.waylay.io]:  api.waylay.io


Using gateway: https://api.waylay.io
Please provide client credentials for the waylay data client.


> apiKey :  41e9fa5124307d9ebeed7909
> apiSecret :  ········
> Do you want to store these credentials with profile=demo? [Y]:  Y


Credential configuration stored as 
	/Users/thomas/Library/Application Support/Waylay/python_sdk/.profile.demo.json
Please make sure this file is treated securely.
If compromised, _Revoke_ the api-key on the Waylay Console!


Once a profile exists, it can be used until you delete it.

In [4]:
waylay_client = WaylayClient.from_profile('demo')

A client will use the endpoint configurations defined in the tenant (global) settings.

In [5]:
waylay_client.list_root_urls()

{'byoml': 'https://api.waylay.io/ml/v1',
 'query': 'https://api.waylay.io/queries/v1',
 'resources': 'https://api.waylay.io/resources/v1',
 'storage': 'https://api.waylay.io/storage/v1',
 'etl': 'https://api.waylay.io/etl/v1',
 'data': 'https://api.waylay.io/data/v1'}

In [6]:
waylay_client.queries.about.version()

'0.3.0'

you can check connection profile by printing the config object

In [7]:
print(waylay_client.config)

{"credentials": {"type": "client_credentials", "api_key": "41e9fa5124307d9ebeed7909", "api_secret": "********", "gateway_url": "https://api.waylay.io", "accounts_url": null}, "profile": "demo", "settings": {}}


you can override endpoints and save the updates

In [8]:
waylay_client.storage.set_root_url('https://storage.waylay.io')
waylay_client.list_root_urls()

{'byoml': 'https://api.waylay.io/ml/v1',
 'query': 'https://api.waylay.io/queries/v1',
 'resources': 'https://api.waylay.io/resources/v1',
 'storage': 'https://storage.waylay.io',
 'etl': 'https://api.waylay.io/etl/v1',
 'data': 'https://api.waylay.io/data/v1'}

In [9]:
waylay_client.queries.about.version()

'0.3.0'

.. and save it in the stored profile

In [10]:
waylay_client.config.save()

print(WaylayConfig.load('demo'))

{"credentials": {"type": "client_credentials", "api_key": "41e9fa5124307d9ebeed7909", "api_secret": "********", "gateway_url": "https://api.waylay.io", "accounts_url": null}, "profile": "demo", "settings": {"waylay_storage": "https://storage.waylay.io"}}


when not specifying a profile name, the `_default_` profile is used.

In [11]:
# WaylayConfig.delete()
waylay_client = WaylayClient.from_profile()
print(waylay_client.config)

{"credentials": {"type": "client_credentials", "api_key": "fc29ca8f37544723fc39d908", "api_secret": "********", "gateway_url": "https://api-aws-dev.waylay.io", "accounts_url": null}, "profile": "_default_", "settings": {}}


## Managing config profiles

The _WaylayConfig_ class provides the following methods to interact with config profiles.

In [12]:
# list profiles
WaylayConfig.list_profiles()

# delete profiles
WaylayConfig.delete('example_notebook')

# load configuration from profile
config = WaylayConfig.load('demo')
waylay_client = WaylayClient(config)

# .. alternatively ...
WaylayClient.from_profile('demo')

# save (altered) configuration in a profile
config.profile = 'demo_copy'
config.save()
config

waylay configuration not found: /Users/thomas/Library/Application Support/Waylay/python_sdk/.profile.example_notebook.json


<WaylayConfig({"credentials": {"type": "client_credentials", "api_key": "41e9fa5124307d9ebeed7909", "api_secret": "********", "gateway_url": "https://api.waylay.io", "accounts_url": null}, "profile": "demo_copy", "settings": {"waylay_storage": "https://storage.waylay.io"}})>

## Alternative authentication methods
Alternatively explicitely configure a Waylay Client with client credentials or waylay token. (e.g. reading it from you own-defined environment variables)

### Authenticate with client credentials

In [13]:
import os
api_key = os.getenv('WAYLAY_TEST_USER_ID')
api_secret = os.getenv('WAYLAY_TEST_USER_SECRET')
gateway_url = os.getenv('WAYLAY_TEST_GATEWAY_URL',  'https://api-aws-dev.waylay.io')
waylay_client_from_cc = WaylayClient.from_client_credentials(api_key, api_secret, gateway_url=gateway_url)
str(waylay_client_from_cc.config)

'{"credentials": {"type": "client_credentials", "api_key": "fc29ca8f37544723fc39d908", "api_secret": "********", "gateway_url": "https://api-aws-dev.waylay.io", "accounts_url": null}, "profile": "_default_", "settings": {}}'

### Authenticate with a waylay token

In [14]:
# temporary user token generated from the client credentials above
token = waylay_client_from_cc.config.get_valid_token()

In [15]:
token.domain

'staging.dev.waylay.io'

In [16]:
# string representation is the signed token
signed_token_string = str(token)

In [17]:
waylay_client_from_token = WaylayClient.from_token(
    signed_token_string, 
    gateway_url=waylay_client_from_cc.config.gateway_url
)

In [18]:
str(waylay_client_from_token.config)

'{"credentials": {"type": "token", "token": "*********", "gateway_url": "https://api-aws-dev.waylay.io", "accounts_url": null}, "profile": "_default_", "settings": {}}'

Note that clients with a _token credential_ configuration will fail after the token has expired. 

Clients with _client credentials_ will renew the token automatically when it expires.

### saving a profile elsewhere
You can export and reconstruct configurations also manually:

In [19]:
demo_config = WaylayConfig.load('demo')

In [20]:
demo_config_json = demo_config.to_dict(obfuscate=False)
restored_config = WaylayConfig.from_dict(demo_config_json)
restored_config.to_dict()

{'credentials': {'type': 'client_credentials',
  'api_key': '41e9fa5124307d9ebeed7909',
  'api_secret': '********',
  'gateway_url': 'https://api.waylay.io',
  'accounts_url': None},
 'profile': 'demo',
 'settings': {'waylay_storage': 'https://storage.waylay.io'}}

In [21]:
restored_client = WaylayClient(restored_config)

In [22]:
restored_client.list_root_urls()

{'byoml': 'https://api.waylay.io/ml/v1',
 'query': 'https://api.waylay.io/queries/v1',
 'resources': 'https://api.waylay.io/resources/v1',
 'storage': 'https://storage.waylay.io',
 'etl': 'https://api.waylay.io/etl/v1',
 'data': 'https://api.waylay.io/data/v1'}

In [23]:
demo_config_json = demo_config.to_dict(obfuscate=False)

WaylayClient(WaylayConfig.from_dict(demo_config_json))

<WaylayClient(services=[byoml,timeseries,queries,resources,storage,util,etl,data],config={"credentials": {"type": "client_credentials", "api_key": "41e9fa5124307d9ebeed7909", "api_secret": "********", "gateway_url": "https://api.waylay.io", "accounts_url": null}, "profile": "demo", "settings": {"waylay_storage": "https://storage.waylay.io"}})>